# This notebook is for the creation of images which are then turned into videos

#### First import the required modules

Also grab useful globals and initialise matplotlib with nicer parameters

In [1]:
# Show plots in the browser
%matplotlib inline
#%matplotlib notebook

import matplotlib as mpl
import matplotlib.colors as col
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import numpy as np
#from datetime import datetime

from utilities import plotting, utils, fio




### AFAC Outline of weather at fire site

#### 211 topography, sites of interest, surface wind, and transect line

I can just plot all the sites I have since the map will only show those within the desired extent (boundaries E,W,S,N)

#### 223 vertical motion along transect

#### 224 horizontal wind speed along transect

Transects also show the topography as a black area along the bottom, Eventually we will add colour to this to depict fire heat flux (when I have fire data and I can read it)

In [ ]:
def winds_2panel(topog, w, s, u, v, z, lat, lon, 
                 extentname='waroona',
                 transect=1, 
                 vectorskip=10,
                 quiverscale=60,
                 dtime=None,
                 ext='.png'
                ):
    '''
    211 Plot showing contour map and wind speed, along with near sites and transect
    223 plot showing vertical motion along transect
    224 plot showing wind speed along transect
    INPUTS:
        topography, vert motion, wind speed, z,lat,lon, 
        extentname = { 'waroona' | 'sirivan' } choice of two fire locations
        transect = int from 1 to 6 for transect choice
        vectorskip reduces quiver density (may need to fiddle)
        quiverscale changes how long the arrows are (also may need to fiddle)
        dtime is datetime of output 
        ext is the plot extension { '.png' | '.eps' }
    Since the boundaries will be set by the extent, I just plot all the locations in _latlons_ 
    '''
    # datetime timestamp for file,title
    if dtime is None:
        dstamp = "20YYMMDDHHMM"
    else:
        dstamp = dtime.strftime("%Y%m%d%H%M")
    # figure name and location
    pname="figures/%s/winds_outline_X%d/fig_%s%s"%(extentname,transect,dstamp,ext)
    
    # set font sizes
    plotting.init_plots()
    
    # get plot extent, and transect
    extent = plotting._extents_[extentname]
    start,end = plotting._transects_["%s%d"%(extentname,transect)]
    
    plt.figure(figsize=[7,10])
    ax1 = plt.subplot(2,1,1)
    
    # top panel is topography
    plotting.map_topography(extent,topog,lat,lon)
    plt.title('Topography, wind speed '+hourstamp)
    
    # start to end x=[lon0,lon1], y=[lat0, lat1]
    plt.plot([start[1],end[1]],[start[0],end[0], ], '--k', 
             linewidth=2, marker='o', markersize=5)
    
    # add nearby towns
    plotting.map_add_locations(['waroona','yarloop','sirivan','uarbry'], 
                               text=['Waroona', 'Yarloop','Sir Ivan','Uarbry'], 
                               textcolor='lightgrey')
    # add fire ignition
    plotting.map_add_locations(['fire_waroona','fire_sirivan'],
                               text = ['Fire ignition']*2, 
                               color='r', marker='*', textcolor='lightgrey')
    # add pyroCB
    
    
    # Add vectors for winds
    # just surface, and one every 10 points to reduce density
    skip = (slice(None,None,vectorskip),slice(None,None,vectorskip))
    #mlon,mlat = np.meshgrid(lon,lat)
    plt.quiver(lon[skip[1]],lat[skip[0]],u[0][skip],v[0][skip], scale=quiverscale)
    
    ax2 = plt.subplot(2,2,3)
    plotting.transect_w(w,z, lat, lon,start,end,topog=topog)
    
    ax3 = plt.subplot(2,2,4)
    plotting.transect_s(s,z,lat,lon,start,end,topog=topog)
    plt.yticks([])
    
    # Save figure into animation folder with numeric identifier
    
    
# Check winds_2panel 


## EXAMPLE 
## Read sir ivan data
data = fio.read_sirivan([fio._files_sirivan_[0]])
for k,v in data.items():
    print(k, np.shape(v))

print (data['hour'], data['time'])
tstep=0
winds_2panel(data['topog'], data['upward_air_velocity'][tstep], data['wind_speed'][tstep],
             data['x_wind_destaggered'][tstep], data['y_wind_destaggered'][tstep],
             data['zth'][tstep], data['latitude'], data['longitude'], 
             extentname='sirivan')
plt.show()

## Now run this in a loop, creating one figure for each time step

The datestamp will be used to name each figure, hopefully ffmpeg can handle this